<h1 align="center"><font color="yellow">RAG and OpenAI’s Function-Calling for Question-Answering with Langchain</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Link de estudo:

* [RAG OpenAI Q&A Function Calling ang LangChain](https://dipankarmedh1.medium.com/exploring-the-power-of-rag-and-openais-function-calling-for-question-answering-d512c45c56b5)

# <font color="red">Retrieval Augmented Generation for Question-answering</font>

<font color="orange">Simplificando o processo de perguntas e respostas com `RAG` e o método de chamada de função (`Function Calling`) mais recente da `OpenAI`.


Se você está cansado de pesquisar manualmente em `documentos` ou `bancos de dados` para encontrar as informações necessárias e deseja uma maneira mais eficiente de extrair dados de fontes externas, pode estar interessado em usar o `Retrieval Augmented Generation (RAG)`.

A `Geração Aumentada de Recuperação (RAG)` é uma nova abordagem para resposta a perguntas que combina os pontos fortes dos modelos de resposta a perguntas extrativos e generativos. Os modelos `RAG` <font color="pink">primeiro usam um recuperador para identificar um conjunto de documentos relevantes de uma base de conhecimento.</font> <font color="green">Em seguida, eles usam um gerador para criar uma nova resposta baseada nos documentos recuperados e na pergunta original.</font>
</font>

# <font color="red">RAG simplificado | O que é RAG ?</font>

No `RAG`, os dados externos podem vir de qualquer lugar, como um repositório de documentos, bancos de dados ou APIs.

1. O primeiro passo é converter os documentos, bem como a query do usuário, no formato para que possam ser comparados e a `pesquisa por similaridade` possa ser realizada.


2. E para tornar os formatos comparáveis ​​para fazer a pesquisa, uma coleção de documentos (`knowledge hub`) e a query enviada pelo usuário são convertidas em representação numérica, chamada de `Embeddings`, usando modelos de linguagem de Embeddings.


3. Os embeddings de documentos são essencialmente representações numéricas de conceitos em texto, ou podemos dizer vetores. Os embeddings podem ser armazenados em um banco de dados vetorial como `Chroma`, `Weaviate`.


4. Em seguida, com base nos Embeddings criado a partir da query do usuário, um texto similar é identificado na coleção de documentos por uma busca por similaridade no espaço de Embeddings.


5. Em seguida, o `prompt + entered_text` é adicionado ao contexto. Todo o prompt é agora enviado para o `LLM` e como o contexto possui dados externos relevantes junto com o prompt original, a saída do modelo é relevante e precisa.


![Alt text](image.png)

# <font color="red">Q&A application using OpenAI</font>

Agora que temos uma ideia sobre os sistemas `RAG Q&A`, podemos começar a experimentar e tentar construir um aplicativo que aproveite os conceitos `RAG` que discutimos acima. Vamos começar!

## <font color="pink">Extração de texto</font>

Começamos extraindo os dados dos documentos. Aqui vamos usar apenas documentos `PDF`.

Alguns dos pacotes que podem ajudar a extrair texto de documentos são:

* PyPDF2

* pdf2imagem

* Pytesseract

<font color="orange">Ao aproveitar o poder da tecnologia `OCR` e de bibliotecas como `pytesseract` e `pdf2image`, esta função pode ajudar aos usuários a automatizar o processo de extração de dados de texto de documentos digitalizados.</font>

In [1]:
import numpy as np
from pdf2image import convert_from_path
import pytesseract




def extract_data_from_scanned_pdf(file_path: str) -> str:
    data = ""
    config = r'--oem 2 --psm 6'
    images = convert_from_path(file_path)
    for i in range(len(images)):
        img = np.array(images[i])
        data += pytesseract.image_to_string(img, config=config)

    return data

## <font color="pink">Splitting do text em chunks</font>

<font color="orange">Os dados extraídos podem ser grandes demais para serem ingeridos pelo modelo de linguagem. Portanto, o próximo passo é dividir os dados em pedaços (`chunks`) menores.


Esta função pode ser útil para dividir uma grande sequência de dados de texto em pedaços menores e mais gerenciáveis ​​usando o objeto `“CharacterTextSplitter”` do `Langchain` e especificando o tamanho do pedaço desejado.
</font>

In [2]:
from langchain.text_splitter import CharacterTextSplitter

def get_data_chunks(data: str, chunk_size: int):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=5, separator="\n", length_function=len)
    chunks = text_splitter.split_text(data)
    return chunks


## <font color="pink">Criando o knowledge hub ou Database</font>

<font color="orange">Terminamos de dividir os dados em pedaços (`chunks`) menores de texto. Agora precisamos converter esses pedaços em `Embeddings` e armazená-los em um banco de dados vetorial. Podemos usar `Chroma` ou `FAISS` ou qualquer outro banco de dados.</font>

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS


def create_knowledge_hub(chunks: list):
    embeddings = OpenAIEmbeddings()
    knowledge_hub = FAISS.from_texts(chunks, embeddings)
    return knowledge_hub


## <font color="pink">Usando a chain de Q&A para retrieval de Informações</font>

<font color="orange">Em seguida, um modelo de linguagem grande (`LLM`) é usado para fazer perguntas, recuperando a resposta relevante de um determinado dado de texto usando um pipeline de `geração aumentada de recuperação` (RAG).</font>

In [4]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


def get_answer_LLM(
        question: str,
        data: str,
        chunk_size: int = 1000,
        chain_type: str = 'stuff',
    ) -> str:    
    if data == "":
        return ""
    
    chunks = get_data_chunks(data, chunk_size=chunk_size)  # create text chunks
    knowledge_hub = create_knowledge_hub(chunks)  # create knowledge hub


    retriever = knowledge_hub.as_retriever(search_type="similarity",
                                           search_kwargs={"k": 2}
                                          )
    # O método 'RetrievalQA.from_chain_type' é usado para criar um pipeline de RAG. A função usa o método 
    # chain do pipeline RAG para gerar uma resposta à pergunta de entrada.
    chain = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0.0, model_name="text-davinci-003"),
                                        chain_type=chain_type,
                                        retriever=retriever,
                                        return_source_documents=True,
                                       )
    
    result = chain({"query": question})

    return result['result']


# <font color="red">Recuperação de informações usando o recurso Function Calling da OpenAI</font>

## <font color="pink">Respondendo algumas perguntas de documentos</font>

Escrever a função para gerar a mensagem e recuperar os argumentos como resposta do método de chamada de função (`Function Calling`).

In [ ]:
from ast import Dict, List
import openai

def message_generate(data:str) -> List[Dict[str, str]]:
    prompt = f"Please extract information from this given data: {data}." # create the prompt
    messages = [{"role": "user", "content": prompt}] 
    return messages

def function_call(
    data: str,
    functions:List[Dict[str, any]], 
    model:str ="gpt-3.5-turbo-0613", 
    function_call:str="auto"
) -> Dict:
    arguments = {}
    message = message_generate(data) # create the message

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=message,
        functions=functions,
        function_call="auto"
    ) 
    arguments = eval(response.choices[0]["message"]["function_call"]["arguments"]) # convert to dic
    return arguments


Para usar a função do `OpenAI` para automatizar seu processo de `Q&A`, você precisará fornecer um argumento de mensagem com a função e o campo de conteúdo (`prompt`) como entrada. O prompt é o que direciona o modelo com as tarefas a serem realizadas e determina o tipo de informação que será recuperada.

Ao usar a `função OpenAI`, é importante garantir que você forneça os argumentos necessários no formato correto. Os campos obrigatórios podem ser encontrados na documentação da função, que fornece instruções detalhadas sobre como usar a função de forma eficaz.

## <font color="pink">Defina a função/propriedades necessárias</font>

Agora precisamos formatar as funções ou os campos a serem extraídos. Nomeando-os como funções.

In [10]:
functions = [  
              {
                  "name": "get_answers",
                  "description": "Get the answers of the questions asked",
                  "parameters": {
                       "type": "object",
                       "properties": {
                            "What is the date of the lease?": {
                                 "type": "string",
                                 "description": "Date of the lease document",
                             },
                            "What is the length of the tenancy?":{
                                 "type": "string",
                                 "description": "The length of the tenancy"
                             },
                            "What is the address of the property being leased?":{
                                 "type": "string",
                                 "description": "The adress of the property that is being leased"
                             },           
                         },
                        "required": [
                          "What is the date of the lease?",
                          "What is the length of the tenancy?",
                          "What is the address of the property being leased?",
                         ],
                        }
                    }
                ]